In [9]:
import os
import json
import sys
import numpy as np
import random
from smol.io import load_work
from pymatgen.core.structure import Structure
from pymatgen.entries.computed_entries import ComputedStructureEntry
from pymatgen.io.vasp.inputs import Poscar
from ase.db import connect
from ase.io import write
from ase.visualize import view

sys.path.append('../Modules')
from structure_generation import * 
from vasp_misc import *


# Composition Selection

## functions

In [ ]:
import json
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go

def load_compositions(json_file):
    """Load compositions from a JSON file."""
    with open(json_file) as f:
        compositions = json.load(f)
    df = pd.DataFrame(compositions)
    if 'Generation' not in df.columns:
        df['Generation'] = 0
    return df

def filter_compositions(df):
    """Filter compositions to ensure they meet the initial constraints."""
    return df[(df['V'] >= 0.8) & 
              (df['Cr'] >= 0.001) & 
              (df['Ti'] >= 0.001) & 
              (df['W'] >= 0.001) & 
              (df['Zr'] >= 0.001)]

def generate_random_composition():
    """Generate a valid random composition within the specified constraints."""
    min_v = 0.85
    while True:
        V = np.random.uniform(min_v, 1)
        Cr = np.random.uniform(0.001, 0.2)
        Ti = np.random.uniform(0.001, 0.2)
        W = np.random.uniform(0.001, 0.2)
        Zr = np.random.uniform(0.001, 0.2)
        total = V + Cr + Ti + W + Zr
        V /= total
        Cr /= total
        Ti /= total
        W /= total
        Zr /= total
        if V >= min_v:
            return [V, Cr, Ti, W, Zr]

def generate_new_compositions(n, generation):
    """Generate new compositions and return as a DataFrame."""
    new_compositions = []
    for _ in range(n):
        new_compositions.append(generate_random_composition())
    new_df = pd.DataFrame(new_compositions, columns=['V', 'Cr', 'Ti', 'W', 'Zr'])
    new_df['Generation'] = generation
    return new_df

def combine_compositions(df_existing, df_new):
    """Combine existing and new compositions into one DataFrame."""
    return pd.concat([df_existing, df_new], ignore_index=True)

def save_compositions_to_json(df, json_file):
    """Save compositions to a JSON file."""
    compositions_list = df.to_dict(orient='records')
    with open(json_file, 'w') as f:
        json.dump(compositions_list, f, indent=4)

def plot_compositions(df):
    """Plot compositions using Plotly with manual color assignment for each generation."""
    unique_generations = df['Generation'].unique()
    colors = px.colors.qualitative.Set1 + px.colors.qualitative.Set2 + px.colors.qualitative.Set3
    
    if len(unique_generations) > len(colors):
        raise ValueError("Too many generations for the available color palette")

    color_map = {gen: colors[i] for i, gen in enumerate(sorted(unique_generations))}
    df['color'] = df['Generation'].map(color_map)

    fig = px.scatter_matrix(df,
                            dimensions=['V', 'Cr', 'Ti', 'W', 'Zr'],
                            color='color',
                            title='V-Cr-Ti-W-Zr Alloy Compositions by Generation',
                            labels={'V': 'Vanadium', 'Cr': 'Chromium', 'Ti': 'Titanium', 'W': 'Tungsten', 'Zr': 'Zirconium'},
                            height=800)
    
    # Hide the automatic legend entries
    for trace in fig.data:
        trace.showlegend = False

    # Updating the color legend manually
    for gen, color in color_map.items():
        fig.add_trace(go.Scatter(x=[None], y=[None], mode='markers',
                                 marker=dict(size=10, color=color),
                                 legendgroup=str(gen),
                                 showlegend=True,
                                 name=f'Generation {gen}'))
    
    fig.update_traces(marker=dict(size=10, opacity=0.8), selector=dict(mode='markers'))
    fig.show()


## generate compositions and view

In [ ]:
random.seed(42)
pre = 3
post = 4
num_compositions = 10
# Example usage
if __name__ == "__main__":
    # Load and filter existing compositions
    df_existing = load_compositions(f'compositions_gen{pre}.json')
    df_existing = filter_compositions(df_existing)
    
    # Generate new compositions for generation 1
    df_new = generate_new_compositions(num_compositions, post)
    
    # Combine existing and new compositions
    df_combined = combine_compositions(df_existing, df_new)
    
    # Save combined compositions to JSON
    save_compositions_to_json(df_combined, f'compositions_gen{post}.json')
    
    # Plot combined compositions
    plot_compositions(df_combined)


In [10]:
gen_num = 3
json_file = f'../Notebooks/compositions_gen{gen_num}.json'
output_dir = f'../Visualization/Job_Structures/Pre_VASP/VCrTiWZr_Summit/Gen_{gen_num}/Perfect'

# make the output directory
if not os.path.exists(output_dir):
    os.makedirs(output_dir)



In [11]:
supercells = create_supercells_for_compositions(json_file, output_dir, num_structures=3, lattice_parameter=3.01 , supercell_type='prim')

In [5]:
print(len(supercells))

96


In [12]:
# now if generation = 0, let's make vasp relaxation jobs for all of these structures
# first we will get the list of all the .cif files in the structure output directory

gen_num = 3
cif_files = [f for f in os.listdir(output_dir) if f.endswith('.cif')]

# next select only the structure names that contains gen0
gen0_structures = [f for f in cif_files if f'gen{gen_num}' in f]

# now define vasp_job folder
vasp_job_dir = f'../Archived_Vasp_Jobs/VCrTiWZr_Summit/gen{gen_num}'
# check if the vasp_job_dir exists, if not create it
if not os.path.exists(vasp_job_dir):
    os.makedirs(vasp_job_dir)
kpoints = (6,6,6)
# now we will create the vasp job files for all the gen0 structures
for f in gen0_structures:
    cif_file = os.path.join(output_dir, f)
    struct = Structure.from_file(cif_file)

    # define job path 
    vasp_job_path = os.path.join(vasp_job_dir, f.replace('.cif', ''))

    # make the vasp_job_path directory
    if not os.path.exists(vasp_job_path):
        os.makedirs(vasp_job_path)

    # make a vasp job from the structure
    make_vasp_job(struct, vasp_job_path, kpoints_params=kpoints, vol_relax=True)



/home/myless/.mambaforge/envs/chgnet/lib/python3.10/site-packages/pymatgen/io/cif.py:1186: UserWarning: The default value of primitive was changed from True to False in https://github.com/materialsproject/pymatgen/pull/3419. CifParser now returns the cell in the CIF file as is. If you want the primitive cell, please set primitive=True explicitly.
  warnings.warn(


# Generate Defects from pristine systems

## defect functions

In [15]:
import os
import json
import sys
import numpy as np
import random
from smol.io import load_work
from pymatgen.core.structure import Structure
from pymatgen.entries.computed_entries import ComputedStructureEntry
from pymatgen.io.vasp.inputs import Poscar
from ase.db import connect
from ase.io import write
from ase.visualize import view

sys.path.append('../Modules')
from defect_maker import make_defects, return_x_neighbors

# Function to load and sort the structure
def load_and_sort_structure(entry):
    return Structure.from_dict(entry['structure']).get_sorted_structure()

# Function to find N target atoms without overlapping second nearest neighbors
def find_target_atoms(structure, N):
    all_indices = list(range(len(structure)))
    random.shuffle(all_indices)
    target_atoms = []
    neighbor_sets = []

    print(f"All indices: {all_indices}")

    while all_indices and len(target_atoms) < N:
        index = all_indices.pop()
        neighbors, _ = return_x_neighbors(structure, target_atom_index=index, x_neighbor=2, alat=structure.lattice.a)
        
        print(f"Index: {index}, Neighbors: {neighbors}")

        if not any(set(neighbors).intersection(neighbor_set) for neighbor_set in neighbor_sets):
            target_atoms.append(index)
            neighbor_sets.append(set(neighbors))
            print(f"Selected target atom index: {index}")

    return target_atoms if len(target_atoms) == N else None

# Function to select a random neighbor
def select_random_neighbor(structure, target_atom_index, x_neighbor):
    neighbors, _ = return_x_neighbors(structure, target_atom_index, x_neighbor, structure.lattice.a)
    if neighbors:
        return random.choice(neighbors)
    return None

# Function to create and save structures with vacancies
def create_and_save_structures(entries, N, job_path):
    for k, entry in enumerate(entries):
        print(f"Processing entry {k+1}/{len(entries)}...")
        structure = load_and_sort_structure(entry)
        target_atoms = find_target_atoms(structure, N)
        if not target_atoms:
            print(f"No suitable target atoms found for entry {k+1}. Skipping...")
            continue
        print(f"Found target atoms for entry {k+1}: {target_atoms}")

        for n, target_atom_index in enumerate(target_atoms):
            start_structure, _ = make_defects(structure, target_atom_index, target_atom_index)
            if start_structure is None:
                print(f"Failed to create start structure for entry {k+1}, target atom {target_atom_index}.")
                continue

            for x_neighbor in [1, 2, 3]:
                vac_site = select_random_neighbor(structure, target_atom_index, x_neighbor)
                if vac_site is not None:
                    _, end_structure = make_defects(structure, target_atom_index, vac_site)
                    if end_structure is None:
                        print(f"Failed to create end structure for entry {k+1}, target atom {target_atom_index}, vac_site {vac_site}.")
                        continue

                    # Create directory and filenames
                    directory = os.path.join(job_path, f"structure_{k}_vac_site_{n}")
                    os.makedirs(directory, exist_ok=True)
                    print(f"Created directory: {directory}")

                    start_filename = os.path.join(directory, f"structure_{k}_vac_site_{n}_start.vasp")
                    end_filename = os.path.join(directory, f"structure_{k}_vac_site_{n}_end_site_{vac_site}.vasp")

                    # Write the structures to POSCAR files
                    Poscar(start_structure).write_file(start_filename)
                    print(f"Written start structure to {start_filename}")
                    Poscar(end_structure).write_file(end_filename)
                    print(f"Written end structure to {end_filename}")



## use functions to make defects

Here I need to save all the finished perfect jobs as pymatgen computed structure entries

In [ ]:
# Directory containing the .mson files
mcmc_path = '../Visualization/Job_Structures/Pre_VASP/VCrTi_fixed_64'
output_filepath = '../Entries/post_mcmc_vcrti_entries.json'

# Initialize an empty list to store the computed structure entries
computed_entries = []

# Iterate over each file in the directory
for filename in os.listdir(mcmc_path):
    if filename.endswith('.mson'):
        filepath = os.path.join(mcmc_path, filename)
        
        # Load the sample data from the .mson file
        samples = load_work(filepath)
        
        # Get the energies and structures
        energies = samples['SampleContainer'].get_energies()
        # Get the minimum energy and corresponding structure
        min_energy_index = np.argmin(energies)
        min_energy = energies[min_energy_index]

        min_structure = samples['SampleContainer'].get_sampled_structures(indices=[min_energy_index])[0]
        
        # Create a ComputedStructureEntry for the minimum energy structure
        min_entry = ComputedStructureEntry(structure=min_structure, energy=min_energy)
        
        # Append the entry to the list
        computed_entries.append(min_entry)

# Convert the list of entries to a JSON serializable format
entries_dict = [entry.as_dict() for entry in computed_entries]

# Save the list as a JSON file
with open(output_filepath, 'w') as f:
    json.dump(entries_dict, f, indent=4)

print(f"Saved computed entries to {output_filepath}")

# Load the computed entries from the JSON file
refined_entries = json.load(open(output_filepath, 'r'))

# Path to save the VASP job structures
vasp_job_path = '../Archived_Vasp_Jobs/VCrTi_Vacancies_T3'

# Create and save structures with vacancies
create_and_save_structures(refined_entries, N=3, job_path=vasp_job_path)

Now create vasp vacancy calculation start jobs (isif = 3) from these folders

In [ ]:

def create_vasp_jobs_from_vasp_files(vasp_files_dir, vasp_job_path, kpoints_params, vol_relax=False, incar_params=None):
    for subdir in os.listdir(vasp_files_dir):
        subdir_path = os.path.join(vasp_files_dir, subdir)
        if os.path.isdir(subdir_path):
            for filename in os.listdir(subdir_path):
                if filename.endswith('.vasp'):
                    filepath = os.path.join(subdir_path, filename)
                    job_name = os.path.splitext(filename)[0]
                    job_folder = os.path.join(vasp_job_path, job_name)
                    os.makedirs(job_folder, exist_ok=True)
                    
                    structure = Poscar.from_file(filepath).structure
                    
                    make_vasp_job(supercell=structure, job_path=job_folder, kpoints_params=kpoints_params, vol_relax=vol_relax, incar_params=incar_params)
                    #make_slurm_file(job_path=job_folder, i=job_name,num_gpus=1,omp_threads=10) # comment out if you are on summit
                    print(f"Created VASP job for {filename} in {job_folder}")

# Directory containing the .vasp files in subdirectories
vasp_files_dir = '../Archived_Vasp_Jobs/VCrTi_Vacancies_T3'
vasp_job_path = '../Archived_Vasp_Jobs/VCrTi_Vacancies_T3_jobs'
kpoints_params = (6, 6, 6)

# Create VASP jobs from .vasp files in subdirectories
create_vasp_jobs_from_vasp_files(vasp_files_dir, vasp_job_path, kpoints_params, vol_relax=True) # change vol_relax to True for start points 

Created VASP job for structure_3_vac_site_2_start.vasp in ../Archived_Vasp_Jobs/VCrTi_Vacancies_T3_jobs/structure_3_vac_site_2_start
Created VASP job for structure_3_vac_site_2_end_site_35.vasp in ../Archived_Vasp_Jobs/VCrTi_Vacancies_T3_jobs/structure_3_vac_site_2_end_site_35
Created VASP job for structure_3_vac_site_2_end_site_14.vasp in ../Archived_Vasp_Jobs/VCrTi_Vacancies_T3_jobs/structure_3_vac_site_2_end_site_14
Created VASP job for structure_3_vac_site_2_end_site_5.vasp in ../Archived_Vasp_Jobs/VCrTi_Vacancies_T3_jobs/structure_3_vac_site_2_end_site_5
Created VASP job for structure_2_vac_site_0_end_site_3.vasp in ../Archived_Vasp_Jobs/VCrTi_Vacancies_T3_jobs/structure_2_vac_site_0_end_site_3
Created VASP job for structure_2_vac_site_0_end_site_40.vasp in ../Archived_Vasp_Jobs/VCrTi_Vacancies_T3_jobs/structure_2_vac_site_0_end_site_40
Created VASP job for structure_2_vac_site_0_end_site_43.vasp in ../Archived_Vasp_Jobs/VCrTi_Vacancies_T3_jobs/structure_2_vac_site_0_end_site_43
C